ils en ont 92 features pour entraîner groupe 52

ordinal encoding : 

piezo_obtention_mode
1. Valeur mesurée
2. Mode d'obtention inconnu
3. Valeur reconstituée

piezo_status
1. Donnée contrôlée niveau 2
2. Donnée contrôlée niveau 1
3. Donnée brute
4. Donnée interprétée

piezo_measure_nature_code
1. 0
2. N
3. nan
4. I
5. D
6. S

hydro_status_label
1. Donnée validée
2. Donnée pré-validée
3. Donnée brute
4. Donnée corrigée

hydro_qualification_code
1. 20
2. 12
3. 16




one hot : 
piezo_station_department_code
hydro_hydro_quantity_elab


## Setup

In [1]:

# Import libraries for data manipulation
import pandas as pd
from summarytools import dfSummary

import numpy as np

from icecream import ic
#ic(my_var) : fait un joli print direct

from ydata_profiling import ProfileReport

# import latexify

# Import libraries for data visualization
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats("svg")

plt.style.use('seaborn-v0_8-whitegrid')

import seaborn as sns

import copy as cp

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

from statsmodels.graphics.gofplots import ProbPlot

# Import libraries for building linear regression model
from statsmodels.formula.api import ols

# Import the required function
from scipy.stats import pearsonr

from sklearn.preprocessing import OrdinalEncoder

from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb


import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

# Import library for preparing data
from sklearn.model_selection import train_test_split

# Import library for data preprocessing
from sklearn.preprocessing import MinMaxScaler

from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.stats.diagnostic import het_white
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

# Plot q-q plot of residuals
import pylab
import scipy.stats as stats

## R2
from sklearn.metrics import r2_score


from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")


####################
# Fonctions : 



def corr_plot(df, upper_tri=None, threshold=None):
    
    plt.figure(figsize = (12, 8))
    cmap = sns.diverging_palette(230, 20, as_cmap = True)

    corr = df.corr()
    mask = None

    if threshold != None:
        corr = corr.mask(np.abs(corr) < threshold, np.nan)

    if upper_tri:
        mask = np.tril(np.ones_like(corr, dtype=bool)) # affichera que le triangle supérieur

    sns.heatmap(corr, mask=mask, annot = True, fmt = '.2f', cmap = cmap)
    plt.show()

    return corr

# RMSE
# @latexify.function
def rmse(predictions, targets):
    return np.sqrt(((targets - predictions) ** 2).mean())

# MAPE
# @latexify.function
def mape(predictions, targets):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# MAE
# @latexify.function
def mae(predictions, targets):
    return np.mean(np.abs((targets - predictions)))

# Function to check VIF (TOUTES LES FEATURES DOIVENT ETRE EN FLOAT !!)
def checking_vif(train):
    vif = pd.DataFrame()
    vif["feature"] = train.columns

    # Calculating VIF for each feature
    vif["VIF"] = [
        variance_inflation_factor(train.values, i) for i in range(len(train.columns))
    ]
    return vif


In [2]:
df = pd.read_csv("./data/X_train_Hi5.csv")
df = df.sample(n=500_000, random_state=42)
df.head()

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
2613083,3067028,33,Fri Jun 28 07:31:38 CEST 2024,123.0,Gironde,33517,LE PORT (SOUSSANS-33),['334AG04'],3.59,07795X0106/PZEM9,...,NaN,NaN,NaN,NaN,19,1692.0,23100,6.3,18.5,Average
1268326,1334103,49,Fri Jan 12 10:28:05 CET 2024,93.5,Maine-et-Loire,49194,FONTAINE-MILONJurassique,['139AP14'],40.00,04552X0111/PZ2,...,-0.281423,47.471283,49194.0,18620902.0,8.8,5815.0,23560,8.8,12.5,Average
234393,246594,27,Mon Sep 23 19:38:51 CEST 2024,12.0,Eure,27391,PUITS DE L'ANCIENNE GARE (MARCILLY-SUR-EURE),['121AS01'],75.00,01807X0051/S1,...,1.365070,48.713810,28404.0,29117529.0,1.3,1588.0,25340,16.6,15.9,High
1670802,1756732,44,Fri Jun 28 07:31:38 CEST 2024,8.0,Loire-Atlantique,44025,Forage de LA RIVIERE,['104AB01'],5.00,04503X0068/PZ2,...,-1.881925,47.275110,44045.0,8572341.0,1,4005.0,23620,18.6,19.8,Average
2804886,3267483,77,Thu Sep 19 14:26:45 CEST 2024,15.0,Seine-et-Marne,77341,PIÉZOMÈTRE DE NOYEN-SUR-SEINE (NOYEN-SUR-SEINE...,['928AC01'],57.50,02606X0112/G5,...,NaN,NaN,NaN,NaN,0,384.0,24270,0,8.3,Very Low


In [3]:
target = "piezo_groundwater_level_category"

## Preprocessing

In [4]:
# On tolère 19% de nan dans les colonnes
for proportion_nan_prct in [19]:
    new_df = cp.deepcopy(df)
    for col in new_df.columns:
        if new_df[col].isnull().sum() * 100 / len(df) > proportion_nan_prct:
            new_df.drop(col, axis=1, inplace=True)

# new_df.info()

# Gérer les duplicatas
new_df.drop_duplicates(inplace=True)

# Drop les features indépendantes qui sont ultra corrélées entre elles (id...)
features_to_drop = ["piezo_station_commune_code_insee",
                    "piezo_station_pe_label",
                    "piezo_station_bdlisa_codes",
                    "piezo_station_bss_code",
                    "piezo_station_commune_name",
                    "piezo_station_bss_id",
                    "piezo_bss_code",
                    "piezo_station_update_date",
                    "piezo_qualification",
                    "piezo_continuity_code",
                    "piezo_continuity_name",
                    "piezo_producer_name",
                    "piezo_measure_nature_name",
                    "meteo_name",
                    "hydro_station_code",
                    "hydro_method_code",
                    "hydro_method_label",
                    "insee_med_living_level",
                    "meteo_id",
                    "hydro_qualification_label",
                    "hydro_status_code",
                    "piezo_station_department_name"]

features = new_df.drop(features_to_drop, axis=1)
# features.info()

num_col_features =      ["piezo_station_investigation_depth", 
                        "piezo_station_altitude", 
                        "piezo_station_longitude", 
                        "piezo_station_latitude", 
                        "piezo_producer_code", 
                        "meteo_latitude", 
                        "meteo_longitude", 
                        "meteo_altitude", 
                        "meteo_rain_height", 
                        "meteo_temperature_min", 
                        "meteo_time_tn", 
                        "meteo_temperature_max", 
                        "meteo_time_tx", 
                        "meteo_temperature_avg", 
                        "meteo_temperature_avg_threshold",
                        "meteo_frost_duration", 
                        "meteo_amplitude_tn_tx", 
                        "meteo_temperature_avg_tntm", 
                        "meteo_evapotranspiration_grid", 
                        "distance_piezo_meteo", 
                        "hydro_observation_result_elab", 
                        "hydro_longitude",
                        "hydro_latitude", 
                        "distance_piezo_hydro", 
                        "prelev_other_volume_sum", 
                        "insee_%_agri", 
                        "insee_pop_commune", 
                        "insee_%_ind", 
                        "insee_%_const"]

cat_col_features = ["piezo_station_department_code",  
                    "piezo_obtention_mode", 
                    "piezo_status", 
                    "piezo_measure_nature_code", 
                    "hydro_status_label", 
                    "hydro_qualification_code",  
                    "hydro_hydro_quantity_elab"]


# On traite le dataframe features entier
new_features = cp.deepcopy(features)

new_features[num_col_features] = new_features[num_col_features].replace('N/A - division par 0', np.nan)
new_features[num_col_features] = new_features[num_col_features].replace('nan', np.nan)

for col in num_col_features:
    new_features[col].astype("float")

# imputation des valeurs numériques manquantes (np.nan) : 

imputer_mean_posterior = IterativeImputer(random_state=42, sample_posterior=True)

imputer_mean_posterior.fit(new_features[num_col_features])
values_imputed = imputer_mean_posterior.transform(new_features[num_col_features])

new_features[num_col_features] = values_imputed

# # vérif plus de nan en numérique
# nan_count = new_features[num_col_features].isna().sum()
# print(nan_count)
# total_nan_count = new_features[num_col_features].isna().sum().sum()
# print(total_nan_count)

# Gérer les 3 features datetime

new_features['DATE_piezo_measurement_date'] = pd.to_datetime(new_features['piezo_measurement_date'])

new_features['year_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.year
new_features['month_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.month
new_features['day_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.day
new_features = new_features.drop(['piezo_measurement_date'], axis=1)

new_features['DATE_meteo_date'] = pd.to_datetime(new_features['meteo_date'])

new_features['year_meteo_date'] = new_features['DATE_meteo_date'].dt.year
new_features['month_meteo_date'] = new_features['DATE_meteo_date'].dt.month
new_features['day_meteo_date'] = new_features['DATE_meteo_date'].dt.day
new_features = new_features.drop(['meteo_date'], axis=1)


new_features['DATE_hydro_observation_date_elab'] = pd.to_datetime(new_features['hydro_observation_date_elab'])

new_features['year_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.year
new_features['month_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.month
new_features['day_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.day
new_features = new_features.drop(['hydro_observation_date_elab'], axis=1)


new_features_2 = cp.deepcopy(new_features) # au cas où je fais un truc nul


# target_feature = new_features_2[target]

# Ordinal encoding :

ord_cols = ["piezo_obtention_mode", 
            "piezo_status", 
            "piezo_measure_nature_code", 
            "hydro_status_label",
            "hydro_qualification_code",
            "piezo_groundwater_level_category"]

for col in ord_cols:

    temp = new_features_2[col]
    temp = pd.DataFrame(temp)

    ordinal_encoder = OrdinalEncoder()
    ordinal_encoder.fit(temp)
    ordinal_enc = ordinal_encoder.transform(temp)

    new_features_2['ORDINAL_' + col] = ordinal_enc


new_features_2 = new_features_2.drop(ord_cols, axis=1)


# Onehot encoding
onehot_cols = ["piezo_station_department_code", "hydro_hydro_quantity_elab"]
new_features_2 = pd.get_dummies(new_features_2, columns=onehot_cols, drop_first=True)
# new_features_2[target] = target_feature # tout est onehot encodé sauf la target var

# Drop les colonnes avec "DATE" (datetime qui sert plus à rien) :

date_cols = ["DATE_piezo_measurement_date", "DATE_meteo_date", "DATE_hydro_observation_date_elab"]
new_features_2 = new_features_2.drop(date_cols, axis=1)


# On cast toutes les features (maintenant toutes encodées donc numérique ou booléen) 
# en float

for col in new_features_2.columns:
    new_features_2[col] = new_features_2[col].astype("float64")


# Drop row_index car sert à rien

new_features_2 = new_features_2.drop("row_index", axis=1)

# Drop encore na (jsp pourquoi ils sont revenus)
new_features_2 = new_features_2.dropna()
print(new_features_2.isnull().sum().sum())




0


In [28]:
pd.set_option('display.max_columns', None)
new_features_2.head()

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_producer_code,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,meteo_temperature_min,meteo_time_tn,meteo_temperature_max,meteo_time_tx,meteo_temperature_avg,meteo_temperature_avg_threshold,meteo_frost_duration,meteo_amplitude_tn_tx,meteo_temperature_avg_tntm,meteo_evapotranspiration_grid,distance_piezo_meteo,hydro_observation_result_elab,hydro_longitude,hydro_latitude,distance_piezo_hydro,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_%_ind,insee_%_const,year_piezo_measurement,month_piezo_measurement,day_piezo_measurement,year_meteo_date,month_meteo_date,day_meteo_date,year_hydro_observation_date_elab,month_hydro_observation_date_elab,day_hydro_observation_date_elab,ORDINAL_piezo_obtention_mode,ORDINAL_piezo_status,ORDINAL_piezo_measure_nature_code,ORDINAL_hydro_status_label,ORDINAL_hydro_qualification_code,ORDINAL_piezo_groundwater_level_category,piezo_station_department_code_02,piezo_station_department_code_03,piezo_station_department_code_04,piezo_station_department_code_05,piezo_station_department_code_06,piezo_station_department_code_07,piezo_station_department_code_08,piezo_station_department_code_09,piezo_station_department_code_10,piezo_station_department_code_11,piezo_station_department_code_12,piezo_station_department_code_13,piezo_station_department_code_14,piezo_station_department_code_15,piezo_station_department_code_16,piezo_station_department_code_17,piezo_station_department_code_18,piezo_station_department_code_19,piezo_station_department_code_21,piezo_station_department_code_22,piezo_station_department_code_23,piezo_station_department_code_24,piezo_station_department_code_25,piezo_station_department_code_26,piezo_station_department_code_27,piezo_station_department_code_28,piezo_station_department_code_29,piezo_station_department_code_2A,piezo_station_department_code_2B,piezo_station_department_code_30,piezo_station_department_code_31,piezo_station_department_code_32,piezo_station_department_code_33,piezo_station_department_code_34,piezo_station_department_code_35,piezo_station_department_code_36,piezo_station_department_code_37,piezo_station_department_code_38,piezo_station_department_code_39,piezo_station_department_code_40,piezo_station_department_code_41,piezo_station_department_code_42,piezo_station_department_code_43,piezo_station_department_code_44,piezo_station_department_code_45,piezo_station_department_code_46,piezo_station_department_code_47,piezo_station_department_code_48,piezo_station_department_code_49,piezo_station_department_code_50,piezo_station_department_code_51,piezo_station_department_code_52,piezo_station_department_code_53,piezo_station_department_code_54,piezo_station_department_code_55,piezo_station_department_code_56,piezo_station_department_code_57,piezo_station_department_code_58,piezo_station_department_code_59,piezo_station_department_code_60,piezo_station_department_code_61,piezo_station_department_code_62,piezo_station_department_code_63,piezo_station_department_code_64,piezo_station_department_code_65,piezo_station_department_code_66,piezo_station_department_code_67,piezo_station_department_code_68,piezo_station_department_code_69,piezo_station_department_code_70,piezo_station_department_code_71,piezo_station_department_code_72,piezo_station_department_code_73,piezo_station_department_code_74,piezo_station_department_code_75,piezo_station_department_code_76,piezo_station_department_code_77,piezo_station_department_code_78,piezo_station_department_code_79,piezo_station_department_code_80,piezo_station_department_code_81,piezo_station_department_code_82,piezo_station_department_code_83,piezo_station_department_code_84,piezo_station_department_code_85,piezo_station_department_code_86,piezo_station_department_code_87,piezo_station_department_code_88,piezo_station_department_code_89,piezo_station_department_code_90,piezo_station_department_code_

In [5]:
new_features_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480041 entries, 2613083 to 2374925
Columns: 138 entries, piezo_station_investigation_depth to hydro_hydro_quantity_elab_QmM
dtypes: float64(138)
memory usage: 509.1 MB


### Extraire saison

In [6]:
cols = new_features_2.columns

result = [col for col in cols if "month" in col]
print(result)

new_features_3 = cp.deepcopy(new_features_2)

new_features_3 = new_features_3.drop(new_features_3[~new_features_3["month_meteo_date"].isin([6, 7, 8, 9])].index)

['month_piezo_measurement', 'month_meteo_date', 'month_hydro_observation_date_elab']


In [18]:
dfSummary(new_features_3[["month_meteo_date", "month_hydro_observation_date_elab"]])

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,month_meteo_date[float64],1. 6.02. 7.03. 8.04. 9.0,524 (26.2%)516 (25.8%)494 (24.7%)467 (23.3%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABcCAYAAAB5jMeAAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB8klEQVR4nO3bQWrbYBSF0V/FhGYiYQxejBfRxXYR2o0x1iAJmSiDTtuUxOnV899zVvAGHzIyusO6rg0Svm19AP8PsREztNYeW2sPWx/yF6/ruj5vfQS32R0Ohx/jOO63PuQ9y7JchmH4Kbj7thvHcX86nZ6naXrZ+pjfuV6v3+d53p/P54fWmtju2K611qZpejkej09bH/OOx60P4HZeEIgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNmF1rvz7j2fqQP6l8Gx+zW5blMs/zvhX+jGdZlktr7XXrO7iNz8KJGUz5SPGCQMw9/Iz6Ce1E+XWVZVU/Sq+rLKv6cg/rqrJ/yfAxXhCIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixEVN6ylf1Lj6n/JTPjK8fNgjEmPIRU/nJ5onWmbLrKquq/pRcV1lV9anyuqrk2zGf509dYsRGjNiIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSUXFdVu4evUXZdZVXVHxsEYqyriKn4ZPNE61S5dZVVVb9KrausqvpWcV1V6q2Yr+NPXWLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzElFpXVbmDf6Pcusqqql82CMRYVxHjBYGYN7NY2Lk7NWFhAAAAAElFTkSuQmCC"">",0(0.0%)
2,month_hydro_observation_date_elab[float64],1. 6.02. 7.03. 8.04. 9.0,524 (26.2%)516 (25.8%)494 (24.7%)467 (23.3%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABcCAYAAAB5jMeAAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB8klEQVR4nO3bQWrbYBSF0V/FhGYiYQxejBfRxXYR2o0x1iAJmSiDTtuUxOnV899zVvAGHzIyusO6rg0Svm19AP8PsREztNYeW2sPWx/yF6/ruj5vfQS32R0Ohx/jOO63PuQ9y7JchmH4Kbj7thvHcX86nZ6naXrZ+pjfuV6v3+d53p/P54fWmtju2K611qZpejkej09bH/OOx60P4HZeEIgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNmF1rvz7j2fqQP6l8Gx+zW5blMs/zvhX+jGdZlktr7XXrO7iNz8KJGUz5SPGCQMw9/Iz6Ce1E+XWVZVU/Sq+rLKv6cg/rqrJ/yfAxXhCIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixEVN6ylf1Lj6n/JTPjK8fNgjEmPIRU/nJ5onWmbLrKquq/pRcV1lV9anyuqrk2zGf509dYsRGjNiIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSUXFdVu4evUXZdZVXVHxsEYqyriKn4ZPNE61S5dZVVVb9KrausqvpWcV1V6q2Yr+NPXWLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzElFpXVbmDf6Pcusqqql82CMRYVxHjBYGYN7NY2Lk7NWFhAAAAAElFTkSuQmCC"">",0(0.0%)


## Data manipulation

In [6]:
def get_PCA(X_scaled):
    # PCA pour réduire la dimension

    pca = PCA()
    pca.set_output(transform="pandas")
    comp = pca.fit(X_scaled)

    plt.plot(np.cumsum(comp.explained_variance_ratio_))
    plt.grid()
    plt.xlabel('Number of Principal Components')
    plt.ylabel('Explained Variance')
    sns.despine()

    pca = PCA(n_components=15).fit(X_scaled)    # seuil choisi à partir du plot
    X_transformed = pca.transform(X_scaled)

    X_transformed = pd.DataFrame(X_transformed)

    return X_transformed


In [7]:
# Separate the dependent variable and indepedent variables

ord_target = "ORDINAL_" + target

Y = new_features_3[ord_target]

X = new_features_3.drop(columns=ord_target)


# Robust Scaling de X

from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

transformer = RobustScaler()
transformer.set_output(transform="pandas")
transformer.fit(X)
X_scaled = transformer.transform(X)

# #PCA : 
# X_transformed = get_PCA(X_scaled)


# Add the intercept term
X_scaled = sm.add_constant(X_scaled)   # ou X_transformed si PCA

# splitting the data in 70:30 ratio of train to test data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size = 0.30, random_state=42)

In [8]:
X_train

,const,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_producer_code,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,piezo_station_department_code_85,piezo_station_department_code_86,piezo_station_department_code_87,piezo_station_department_code_88,piezo_station_department_code_89,piezo_station_department_code_90,piezo_station_department_code_91,piezo_station_department_code_93,piezo_station_department_code_95,hydro_hydro_quantity_elab_QmM
1516397,1.0,-0.108844,-0.170764,-0.418619,0.280006,0.990302,-0.418963,0.352493,-0.104938,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1474990,1.0,-0.081633,-0.256944,-1.472654,0.478877,0.989224,-1.447055,0.491976,-0.123457,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
633835,1.0,0.299320,-0.635694,0.514941,-0.796793,-0.006466,0.507628,-0.783001,-0.549383,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1459772,1.0,-0.040816,0.750000,-1.370099,0.551814,0.989224,-1.388861,0.566546,0.234568,4.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1322420,1.0,0.870748,0.166667,0.173440,0.709305,-0.066810,0.186967,0.725839,0.197531,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547019,1.0,0.610884,0.118056,-0.237971,0.846792,-0.009698,-0.343996,0.800725,0.080247,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
441956,1.0,-0.451020,-0.182083,-0.154093,-0.681540,-0.003233,-0.177781,-0.675929,0.259259,41.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
434464,1.0,-0.059320,-0.034722,0.319491,0.937940,-0.037716,0.348220,0.925113,-0.067901,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1321758,1.0,0.027211,4.935417,0.290782,-0.162186,-0.110991,0.295481,-0.167951,5.320988,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X_test

,const,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_producer_code,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,...,piezo_station_department_code_85,piezo_station_department_code_86,piezo_station_department_code_87,piezo_station_department_code_88,piezo_station_department_code_89,piezo_station_department_code_90,piezo_station_department_code_91,piezo_station_department_code_93,piezo_station_department_code_95,hydro_hydro_quantity_elab_QmM
570491,1.0,0.326531,1.903472,-0.037033,-0.088524,-7.435345e-02,-0.039944,-0.080508,2.166667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1577148,1.0,0.153061,0.165833,0.152659,0.497594,-7.543103e-03,0.107653,0.475793,0.067901,1.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1462741,1.0,1.972789,3.493056,0.907402,-0.331672,-1.120690e-01,0.876517,-0.308024,3.228395,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1460749,1.0,4.081633,0.939097,1.246639,0.600980,2.478448e-02,1.205709,0.627561,0.679012,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1420072,1.0,-0.375646,0.763889,-0.704765,0.618603,-9.698276e-03,-0.695866,0.646691,1.401235,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509279,1.0,0.136054,5.465278,0.560883,0.140735,-1.099138e-01,0.521654,0.153490,1.641975,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
573067,1.0,1.442177,0.006944,-0.342984,-0.497712,2.909483e-02,-0.311680,-0.433590,-0.370370,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
592554,1.0,-0.468027,-0.055069,-0.368667,0.086254,2.571121e+10,-0.403912,0.220308,-0.469136,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1435735,1.0,0.227891,-0.006944,-0.279821,0.462638,2.448276e+10,-0.265912,0.479420,0.080247,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Feature Engineering & Selection

### VIF

In [140]:
vif = checking_vif(X_train)
vif

,feature,VIF
0,const,317.523070
1,piezo_station_investigation_depth,1.349110
2,piezo_station_altitude,4.482999
3,piezo_station_longitude,1753.161725
4,piezo_station_latitude,1367.954093
...,...,...
133,piezo_station_department_code_90,1.437472
134,piezo_station_department_code_91,2.978994
135,piezo_station_department_code_93,1.481055
136,piezo_station_department_code_95,1.824318


On vire les features VIF > 5

In [141]:
vif[vif["VIF"] < 5]

,feature,VIF
1,piezo_station_investigation_depth,1.349110
2,piezo_station_altitude,4.482999
5,piezo_producer_code,3.891758
8,meteo_altitude,4.581016
9,meteo_rain_height,1.027756
...,...,...
133,piezo_station_department_code_90,1.437472
134,piezo_station_department_code_91,2.978994
135,piezo_station_department_code_93,1.481055
136,piezo_station_department_code_95,1.824318


In [142]:
features_vif = vif[vif["VIF"] < 5]["feature"].to_numpy()

X_train_2 = X_train[features_vif]
X_test_2 = X_test[features_vif]
X_train_2

,piezo_station_investigation_depth,piezo_station_altitude,piezo_producer_code,meteo_altitude,meteo_rain_height,meteo_temperature_min,meteo_time_tn,meteo_temperature_max,meteo_evapotranspiration_grid,hydro_observation_result_elab,distance_piezo_hydro,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_%_ind,insee_%_const,ORDINAL_piezo_obtention_mode,ORDINAL_piezo_status,ORDINAL_piezo_measure_nature_code,ORDINAL_hydro_status_label,ORDINAL_hydro_qualification_code,piezo_station_department_code_03,piezo_station_department_code_04,piezo_station_department_code_05,piezo_station_department_code_06,piezo_station_department_code_07,piezo_station_department_code_08,piezo_station_department_code_09,piezo_station_department_code_10,piezo_station_department_code_12,piezo_station_department_code_13,piezo_station_department_code_15,piezo_station_department_code_18,piezo_station_department_code_19,piezo_station_department_code_21,piezo_station_department_code_22,piezo_station_department_code_23,piezo_station_department_code_25,piezo_station_department_code_30,piezo_station_department_code_31,piezo_station_department_code_32,piezo_station_department_code_38,piezo_station_department_code_39,piezo_station_department_code_42,piezo_station_department_code_43,piezo_station_department_code_46,piezo_station_department_code_48,piezo_station_department_code_51,piezo_station_department_code_52,piezo_station_department_code_53,piezo_station_department_code_54,piezo_station_department_code_55,piezo_station_department_code_58,piezo_station_department_code_61,piezo_station_department_code_63,piezo_station_department_code_64,piezo_station_department_code_65,piezo_station_department_code_69,piezo_station_department_code_70,piezo_station_department_code_71,piezo_station_department_code_73,piezo_station_department_code_74,piezo_station_department_code_75,piezo_station_department_code_78,piezo_station_department_code_81,piezo_station_department_code_82,piezo_station_department_code_83,piezo_station_department_code_84,piezo_station_department_code_87,piezo_station_department_code_88,piezo_station_department_code_89,piezo_station_department_code_90,piezo_station_department_code_91,piezo_station_department_code_93,piezo_station_department_code_95,hydro_hydro_quantity_elab_QmM
8573,-0.170629,1.173611,-1.120690e-01,1.10000,0.250000,-3.510638e-01,3.363071e+00,-0.707547,-0.555556,-0.205681,-0.333969,0.608706,0.250000,0.247380,-0.176471,1.614604,0.0,0.0,1.000000,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2766621,-0.457343,-0.493958,2.909483e-02,-0.50000,4.062500,2.659574e-01,-1.369295e-01,1.160377,0.703704,-0.195251,-0.032261,2.561342,1.612069,-0.223115,-0.390374,0.308316,0.0,0.0,1.000000,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2773464,0.030769,0.159722,-6.465517e-03,2.98125,1.125000,3.085106e-01,-8.589212e-01,0.339623,0.518519,-0.205264,-0.951988,38.476701,0.931034,-0.342551,-0.390374,-0.300203,0.0,0.0,0.000000,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
970406,0.618182,-0.348681,2.909483e-02,-0.57500,5.625000,-2.446809e-01,-1.068465e+00,-0.566038,-0.481481,0.845392,0.233358,0.667137,-0.301724,0.380997,1.331551,0.081136,0.0,0.0,0.000000,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999664,3.544484,1.927083,-1.088362e-01,2.60625,1

In [143]:
X_test_2

,piezo_station_investigation_depth,piezo_station_altitude,piezo_producer_code,meteo_altitude,meteo_rain_height,meteo_temperature_min,meteo_time_tn,meteo_temperature_max,meteo_evapotranspiration_grid,hydro_observation_result_elab,distance_piezo_hydro,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_%_ind,insee_%_const,ORDINAL_piezo_obtention_mode,ORDINAL_piezo_status,ORDINAL_piezo_measure_nature_code,ORDINAL_hydro_status_label,ORDINAL_hydro_qualification_code,piezo_station_department_code_03,piezo_station_department_code_04,piezo_station_department_code_05,piezo_station_department_code_06,piezo_station_department_code_07,piezo_station_department_code_08,piezo_station_department_code_09,piezo_station_department_code_10,piezo_station_department_code_12,piezo_station_department_code_13,piezo_station_department_code_15,piezo_station_department_code_18,piezo_station_department_code_19,piezo_station_department_code_21,piezo_station_department_code_22,piezo_station_department_code_23,piezo_station_department_code_25,piezo_station_department_code_30,piezo_station_department_code_31,piezo_station_department_code_32,piezo_station_department_code_38,piezo_station_department_code_39,piezo_station_department_code_42,piezo_station_department_code_43,piezo_station_department_code_46,piezo_station_department_code_48,piezo_station_department_code_51,piezo_station_department_code_52,piezo_station_department_code_53,piezo_station_department_code_54,piezo_station_department_code_55,piezo_station_department_code_58,piezo_station_department_code_61,piezo_station_department_code_63,piezo_station_department_code_64,piezo_station_department_code_65,piezo_station_department_code_69,piezo_station_department_code_70,piezo_station_department_code_71,piezo_station_department_code_73,piezo_station_department_code_74,piezo_station_department_code_75,piezo_station_department_code_78,piezo_station_department_code_81,piezo_station_department_code_82,piezo_station_department_code_83,piezo_station_department_code_84,piezo_station_department_code_87,piezo_station_department_code_88,piezo_station_department_code_89,piezo_station_department_code_90,piezo_station_department_code_91,piezo_station_department_code_93,piezo_station_department_code_95,hydro_hydro_quantity_elab_QmM
1720536,0.338462,0.319444,2.909483e-02,-0.39375,0.000,0.063830,-0.195021,-0.622642,-0.481481,-0.167994,-0.113558,-0.112012,-0.301724,-0.290554,-0.235294,-0.170385,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2758629,-0.139161,-0.412500,2.909483e-02,-0.54375,0.000,0.457447,-0.139004,0.971698,1.481481,-0.116260,1.261260,38.808236,-0.258621,0.958639,-0.262032,-0.154158,0.0,0.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2629309,0.520280,0.666667,-1.099138e-01,0.46875,0.250,-0.127660,3.304979,-0.047170,-0.111111,-0.187463,-0.535298,17.404927,-0.215517,0.036871,-0.224599,2.141988,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
506780,-0.402797,-0.731458,2.155343e+10,-0.65625,0.000,1.500000,-0.145228,1.084906,0.814815,-0.011125,0.037898,0.275836,-0.267241,1.529662,-0.363636,-0.348884,0.0,0.0,1.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1311378,-0.529634,-0.570833,-9.698276e-03,-0.60625,0.000,0.414894,-0.763485,0.415094,0.296296,-0.194277,

## Predictive Modeling

### Multinomial Logisitc Regression

In [199]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(multi_class='multinomial', solver ='newton-cg').fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

accuracy_score(y_test, y_pred)

0.22675026123301986

### XGBoost multiclass classifier

#### Test naïf rapide

In [10]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

accuracy_score(y_test, y_pred)

0.6900390435933013

#### GridSearch CV

In [12]:
from scipy.stats import uniform, randint

def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [13]:
params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}


search = RandomizedSearchCV(xgb_model, 
                            param_distributions=params, 
                            random_state=42, 
                            n_iter=70, 
                            cv=3, 
                            verbose=1, 
                            n_jobs=2, 
                            return_train_score=True)

search.fit(X_train, y_train)




Fitting 3 folds for each of 70 candidates, totalling 210 fits


In [146]:
report_best_scores(search.cv_results_, 1)

Model with rank: 1
Mean validation score: 0.330 (std: 0.007)
Parameters: {'colsample_bytree': 0.9078671076075817, 'gamma': 0.17416830222659868, 'learning_rate': 0.3109944455567122, 'max_depth': 5, 'n_estimators': 135, 'subsample': 0.7671784126862315}

